In [3]:
from docplex.mp.model import Model
import plotly.express as px
import pandas

In [4]:
df_costo_cliente_planta = pandas.read_excel("./data.xlsx", sheet_name="COSTO CLIENTE-PLANTA")
df_clientes = pandas.read_excel("./data.xlsx", sheet_name="DEMANDAS CLIENTES")
df_plantas = pandas.read_excel("./data.xlsx", sheet_name="COSTO Y CAPACIDAD PLANTAS")

**Conjuntos**

In [5]:
PLANTAS = df_plantas["Planta"].to_list()
CLIENTES = df_clientes["Cliente"].to_list()

**Parámetros**

In [6]:
COSTO_CLIENTE_PLANTA = df_costo_cliente_planta.set_index(["Planta", "Cliente"])['Costo de asignar un cliente a un planta'].to_dict()
DEMANDA_CLIENTE = df_clientes.set_index(["Cliente"])["Demanda del cliente"].to_dict()
COSTO_ABRIR_PLANTA = df_plantas.set_index(["Planta"])["Costo de abrir planta"].to_dict()
CAPACIDAD_PLANTA = df_plantas.set_index(["Planta"])["Capacidad máxima"].to_dict()

In [7]:
def model_plantas_clientes(min_capacidad: float, tiempo_ejecucion_sec: int) -> float:

    model = Model()

    ## Variables
    x = model.binary_var_dict(PLANTAS, name="x")
    y = model.binary_var_dict([(planta, cliente) for planta in PLANTAS for cliente in CLIENTES], name="y")

    ## Función Objetivo
    costo_abrir_plantas = model.sum(x[planta] * COSTO_ABRIR_PLANTA[planta] for planta in PLANTAS)
    costo_cliente_planta = model.sum(y[(planta, cliente)] * COSTO_CLIENTE_PLANTA[(planta, cliente)] for planta in PLANTAS for cliente in CLIENTES)
    model.minimize(costo_abrir_plantas + costo_cliente_planta)

    ## Restricciones
    for planta in PLANTAS:
        for cliente in CLIENTES:
            model.add_constraint(y[(planta, cliente)] <= x[planta], ctname=f"ct1_{planta}_{cliente}")

    for planta in PLANTAS:
        model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) <= CAPACIDAD_PLANTA[planta], ctname=f"ct2_{planta}")

    for cliente in CLIENTES:
        model.add_constraint(model.sum(y[(planta, cliente)] for planta in PLANTAS) == 1, ctname=f"ct3_{cliente}")

    for planta in PLANTAS:
        model.add_constraint(model.sum(y[(planta, cliente)] for cliente in CLIENTES) >= 3 * x[planta], ctname=f"ct4_{planta}")

    for planta in PLANTAS:
        model.add_constraint(model.sum(y[(planta, cliente)] * DEMANDA_CLIENTE[cliente] for cliente in CLIENTES) >= min_capacidad * CAPACIDAD_PLANTA[planta] * x[planta], ctname=f"ct5_{planta}")

    ## Resolver Modelo
    model.set_time_limit(tiempo_ejecucion_sec)
    solution = model.solve()

    return round(solution.objective_value, 2)

In [8]:
CAPACS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
FOS = []
for capac in CAPACS:
    fo = model_plantas_clientes(capac, 200)
    FOS.append(fo)

In [9]:
fig = px.line(x=CAPACS, y=FOS, markers=True)

fig.update_traces(
    marker=dict(color="red", size=7, symbol="x"),
    line=dict(color="blue", width=3),
)

fig.update_layout(
    coloraxis_showscale=False,
    height=400,
    width=600,
    xaxis={"title": "Capacidades"},
    yaxis={"title": "Función Objetivo"},
    title="Capacidad VS Función Objetivo",
    template="ggplot2",
)

fig.show()
